This file, once complete, will obtain a dataframe that contains every trade that occured during the 2022-23 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [1]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains the cost per save of NHL Goalies for the 2017-18 season. So I am going to scrape the cap friendly website (Cost Per Save tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed because the table only expands to its full length once you scroll down.

In [2]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/cost-per-save/2018/season"
driver.get(url)

# Get initial page height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]



C:\Users\nsofi\AppData\Local\Temp\ipykernel_20268\2996254770.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [3]:
dfattempt.shape
dfattempt.tail(5)

,#,GOALIE,AGE,TYPE,EXPIRY,CAP HIT,P. BONUSES,GP,MIN,SH,SV,SV%,GAA,$/GP,$/MIN,$/SV
74,75,"Lehtonen, Kari",34,NaN,Stnd,UFA,"$5,900,000",37,1944,946,863.0,0.912,2.56,"$159,459","$3,034","$6,836"
75,76,"Crawford, Corey",33,NaN,Stnd,UFA,"$6,000,000",28,1583,842,782.0,0.929,2.27,"$214,285","$3,790","$7,672"
76,77,"Berra, Reto",30,NaN,Stnd,UFA,"$700,000",5,181,95,88.0,0.926,2.32,"$140,000","$3,867","$7,954"
77,78,"Mason, Steve",29,NaN,Stnd,UFA,"$4,100,000",13,684,392,355.0,0.906,3.25,"$315,384","$5,994","$11,549"
78,79,"Läck, Eddie",29,NaN,Stnd,UFA,"$2,750,000",8,343,177,154.0,0.870,4.02,"$343,750","$8,017","$17,857"


In [4]:
dfattempt.to_csv("CF Cost Per Save 2017-18.csv") 